# Объяснение функций Grouper и Agg в Pandas

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/быстрое%20введение%20в%20pandas/Объяснение%20функций%20Grouper%20и%20Agg%20в%20Pandas.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

## Введение

Время от времени полезно сделать шаг назад и посмотреть на новые способы решения старых задач. Недавно, работая над проблемой, я заметил, что в pandas есть функция [`Grouper`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html), которую я никогда раньше не вызывал. Я изучил, как ее можно использовать, и оказалось, что она полезна для того типа сводного анализа, который я обычно выполняю.

> Оригинал статьи Криса по [ссылке](https://pbpython.com/pandas-grouper-agg.html)

В дополнение к ранним функциям pandas с каждым выпуском продолжает предоставлять новые и улучшенные возможности. Например, обновленная функция [`agg`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html) - еще один очень полезный и интуитивно понятный инструмент для обобщения данных.

В этой статье рассказывается, как вы можете использовать функции `Grouper` и `agg` для собственных данных. Попутно я буду включать некоторые советы и приемы, как их использовать наиболее эффективно.

# Группировка данных временных рядов

Pandas берет свое начало в финансовой индустрии, поэтому неудивительно, что у него есть надежные средства для обработки данных временных рядов. Просто посмотрите обширную [документацию по временным рядам](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html), чтобы почувствовать все возможности. 

Рассмотрим пример данных о продажах и некоторые простые операции для получения общих продаж по месяцам, дням, годам и т.д. 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("https://github.com/chris1610/pbpython/blob/master/data/sample-salesv3.xlsx?raw=True")
df.head()

Обратим внимание на типы данных:

In [ ]:
df.info()

Столбец `date` приведем к типу `datetime`:

In [ ]:
df["date"] = pd.to_datetime(df['date'])

In [ ]:
df.dtypes

Прежде чем я продвинусь дальше, полезно познакомиться с [псевдонимами смещения](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) (`Offset Aliases`). Эти строки используются для представления различных временных частот, таких как дни, недели и годы. 

Например, если вы хотите суммировать все продажи по месяцам, то можете использовать функцию `resample`. Особенность использования `resample` заключается в том, что она работает только с индексом. В этом наборе данные не индексируются по столбцу `date`, поэтому `resample` не будет работать без реструктуризации (restructuring). 

Используйте `set_index`, чтобы сделать столбец `date` индексом, а затем выполните `resample`:

In [ ]:
df.set_index('date').resample('M')["ext price"].sum()

Это довольно простой способ суммирования данных, но он усложняется, если вы хотите дополнительно провести группировку. 

Можно посмотреть ежемесячные результаты для каждого клиента:

In [ ]:
df.set_index('date').groupby('name')["ext price"].resample("M").sum()

Это работает, но выглядит немного неуклюжим...

К счастью, `Grouper` упрощает данную процедуру!

Вместо того, чтобы играть с переиндексированием, мы можем использовать обычный синтаксис `groupby`, но предоставить немного больше информации о том, как сгруппировать данные в столбце `date`:

In [ ]:
df.groupby(['name', pd.Grouper(key='date', freq='M')])['ext price'].sum()

Поскольку `groupby` - одна из моих любимых функций, этот подход кажется мне более простым и, скорее всего, останется в моей памяти.

Приятным дополнением является то, что для обобщенния в другом временном интервале, достаточно измените параметр `freq` на один из допустимых [псевдонимов смещения](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects). 

Например, годовая сводка, использующая декабрь в качестве последнего месяца, будет выглядеть так:

In [ ]:
df.groupby(['name', pd.Grouper(key='date', freq='A-DEC')])['ext price'].sum()

Если ваши годовые продажи были не календарными, то данные можно легко изменить, передав параметр `freq`. 

Призываю вас поиграть с разными смещениями, чтобы понять, как это работает. При суммировании данных временных рядов это невероятно удобно! 

Попробуйте реализовать это в `Excel`, что, безусловно, возможно (с использованием сводных таблиц и настраиваемой группировки), но я не думаю, что это так же интуитивно понятно, как в pandas.

## Новая и улучшенная агрегатная функция

В pandas 0.20.0 была добавлена новая функция [`agg`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html), которая значительно упрощает суммирование данных аналогично groupby.

Чтобы проиллюстрировать ее функциональность, предположим, что нам нужно получить сумму в столбцах `ext price` и `quantity` (количество), а также среднее значение `unit price` (цены за единицу). 

Процесс не очень удобный:

In [ ]:
df[["ext price", "quantity"]].sum()

In [ ]:
df["unit price"].mean()

Это работает, но немного беспорядочно... 

Новый `agg` упрощает процесс:

In [ ]:
df[["ext price", "quantity", "unit price"]].agg(['sum', 'mean'])

Хорошие результаты, но включение суммы `unit price` не очень полезно. 

К счастью, мы можем передать словарь в `agg` и указать, какие операции применять к каждому столбцу.

In [ ]:
df.agg({'ext price': ['sum', 'mean'], 'quantity': ['sum', 'mean'], 'unit price': ['mean']})

Я считаю этот подход действительно удобным, когда хочу суммировать несколько столбцов. Раньше я выполнял отдельные вычисления и создавал результирующий `DateFrame` по строке за раз - было утомительно. 

В качестве дополнительного бонуса вы можете определять свои собственные функции. Например, мне часто нужно агрегировать данные и использовать функцию `mode`, которая бы работала с текстом. 

Для своих задач я нашел лямбда-функцию, которая использует `value_counts`:

In [ ]:
get_max = lambda x: x.value_counts(dropna=False).index[0]

Затем, если я хочу включить наиболее часто используемые `sku` (артикулы) в сводную таблицу:

In [ ]:
df.agg({'ext price': ['sum', 'mean'], 'quantity': ['sum', 'mean'], 'unit price': ['mean'], 'sku': [get_max]})

Это довольно круто, но есть одна вещь, которая меня всегда беспокоила в этом подходе: в столбце написано `<lambda>`.

В идеале я хочу указать `most frequent` (*наиболее часто*). Раньше я прыгал через несколько обручей, чтобы произвести переименование, но, работая над этой статьей, я наткнулся на другой подход - явное определение имени лямбда-функции:

In [ ]:
get_max.__name__ = "most frequent"

Теперь, когда я выполняю агрегирование:

In [ ]:
df.agg({'ext price': ['sum', 'mean'], 'quantity': ['sum', 'mean'], 'unit price': ['mean'], 'sku': [get_max]})

Получили гораздо более приятные названия столбцов! Конечно, это мелочь, но я несомненно рад, что понял ее.

В качестве завершающего финального бонуса вот еще один трюк. 

Агрегатная (aggregate) функция, использующая словарь, полезна, но проблема заключается в том, что она не сохраняет порядок. 

Если вы хотите убедиться, что ваши столбцы расположены в определенном порядке, вы можете использовать [`OrderedDict`](https://docs.python.org/3/library/collections.html#collections.OrderedDict):

In [ ]:
import collections
f = collections.OrderedDict([('ext price', ['sum', 'mean']), ('quantity', ['sum', 'mean']), ('sku', [get_max])])
df.agg(f)

## Заключение

Библиотека `pandas` продолжает расти и развиваться с течением времени. Иногда бывает полезно убедиться, что не появилось более простых решений. Функция `Grouper` и обновленная функция `agg` действительно полезны при агрегировании и обобщении данных. 

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>